# Liga cronometro

In [1]:
from time import perf_counter 
inicio = perf_counter()

## Dados de entrada

In [2]:
import pandas as pd

In [3]:
# dados = pd.read_excel('var_comf.xlsx')
dados = pd.read_csv('var_comf.csv')

In [4]:
dados.head()

,Unnamed: 0,ta,tr,vel,rh,et,set,tsens,disc,pd,ps,pts,pmvg,pmvstar
0,1,22.009322,24.918324,-0.532717,67.348226,23.866729,22.825691,-0.055005,0.000000,8.589865,56.258320,-0.323577,-0.037372,-0.002219
1,2,28.867893,22.584649,0.340309,46.376891,25.967170,23.410143,-0.002241,0.002331,19.430093,77.493906,-0.177464,0.031756,-0.081892
2,3,23.173539,23.307901,-0.232876,66.941346,23.440703,22.379056,-0.080457,0.000000,7.755845,62.306999,-0.435236,-0.076198,-0.021384
3,4,24.319086,18.923142,0.539211,42.785042,22.095171,18.155481,-0.485016,0.000000,68.958440,145.361248,-1.491130,-0.664847,-0.864043
4,5,22.550878,24.236664,0.984487,54.880720,23.146186,18.269806,-0.497252,0.000000,194.256672,158.888353,-1.462549,-0.646519,-0.889366


In [5]:
dados.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,150.0,75.500000,43.445368,1.000000,38.250000,75.500000,112.750000,150.000000
ta,150.0,22.904018,3.354546,13.866622,20.681148,22.789670,25.275109,30.141581
tr,150.0,23.866741,3.278672,15.589025,21.689947,23.787180,25.618669,33.048062
vel,150.0,0.249299,0.491247,-1.297657,-0.069528,0.233827,0.568560,1.392951
rh,150.0,60.850833,18.418496,18.156281,47.754310,59.512277,72.333401,116.688260
et,150.0,23.572456,2.629706,15.239219,22.002061,23.495198,25.340375,31.095601
set,150.0,21.076027,3.332182,7.645193,18.749873,21.414899,23.409707,30.654502
tsens,150.0,-0.177733,0.399702,-1.459832,-0.416325,-0.161832,-0.012353,1.501537
disc,150.0,0.045002,0.369286,-1.459832,0.000000,0.000000,0.000000,1.883189
pd,150.0,58.957506,81.406192,2.825121,9.258986,21.859606,77.093134,448.326244


In [6]:
# x_data = dados.drop([' ', 'et', 'set', 'tsens', 'disc', 'pd', 'ps', 'pts', 'pmvg', 'pmvstar'],axis=1)
x_data = dados[['ta', 'tr', 'vel', 'rh']]

In [7]:
y_val = dados['pmvg']

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
X_train, X_test, y_train, y_test = train_test_split(x_data,y_val,test_size=0.3,random_state=101)

### Padronização

In [10]:
from sklearn.preprocessing import MinMaxScaler

In [11]:
scaler = MinMaxScaler()

In [12]:
scaler.fit(X_train)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [13]:
X_train = pd.DataFrame(data=scaler.transform(X_train),columns = X_train.columns,index=X_train.index)

In [14]:
X_test = pd.DataFrame(data=scaler.transform(X_test),columns = X_test.columns,index=X_test.index)

### Rede neural

In [15]:
import tensorflow as tf

In [16]:
import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)

In [17]:
ta  = tf.feature_column.numeric_column('ta')
tr  = tf.feature_column.numeric_column('tr')
vel = tf.feature_column.numeric_column('vel')
rh  = tf.feature_column.numeric_column('rh')

In [18]:
feat_cols = [ta, tr, vel, rh]

In [19]:
print(tf.__version__)

2.0.0


In [20]:
# Função de entrada -> Portar para a versão 2.0.0
input_func = tf.compat.v1.estimator.inputs.pandas_input_fn(x=X_train, y=y_train, batch_size=10, num_epochs=1000, shuffle=True)

In [21]:
# Define estimador
model = tf.estimator.DNNRegressor(hidden_units=[30, 100, 100, 30],feature_columns=feat_cols)

In [22]:
# Treinando o modelo
model.train(input_fn=input_func,steps=25000)

### Aplicando e avaliando o modelo na amostra de treino

In [23]:
len(X_train)

105

In [24]:
len(y_train)

105

In [25]:
predict_input_func_treino = tf.compat.v1.estimator.inputs.pandas_input_fn(x=X_train, batch_size=10, num_epochs=1, shuffle=False)

In [26]:
pred_gen_treino = model.predict(predict_input_func_treino)

In [27]:
predictions = list(pred_gen_treino)

In [28]:
final_preds = []
for pred in predictions:
    final_preds.append(pred['predictions'][0])

In [29]:
from sklearn.metrics import median_absolute_error, r2_score

In [30]:
data_treino = X_train
data_treino['pmvg_est'] = final_preds
data_treino['pmvg'] = y_train
data_treino.head()

,ta,tr,vel,rh,pmvg_est,pmvg
13,0.580840,0.456573,0.675392,0.506063,-0.429385,-0.425102
102,0.676664,0.347735,0.148057,0.275927,-0.003960,-0.119385
67,0.624846,0.490115,0.389848,0.507733,0.099354,0.024559
34,0.489663,0.382493,0.628021,0.800314,-0.511791,-0.556970
98,0.848655,0.516771,0.081011,0.708166,0.541918,0.703390


In [31]:
median_absolute_error(data_treino['pmvg_est'], data_treino['pmvg'])

0.07751880663411365

### Aplicando o modelo na amostra de teste

In [32]:
type(X_test)

pandas.core.frame.DataFrame

In [33]:
len(X_test)

45

In [34]:
type(y_test)

pandas.core.series.Series

In [35]:
len(y_test)

45

In [36]:
predict_input_func_teste = tf.compat.v1.estimator.inputs.pandas_input_fn(x=X_test, batch_size=10, num_epochs=1, shuffle=False)

In [37]:
pred_gen_teste = model.predict(predict_input_func_teste)

In [38]:
pred_teste = list(pred_gen_teste)

In [39]:
final_preds_teste = []
for pred in pred_teste:
    final_preds_teste.append(pred['predictions'][0])

In [40]:
type(final_preds_teste)

list

In [41]:
X_test.head()

,ta,tr,vel,rh
33,0.692642,0.333361,0.705758,0.618035
16,0.553586,0.263474,0.591273,0.280329
43,0.446348,0.448418,0.427409,0.350889
129,0.706284,0.235952,0.671317,0.025447
50,0.323094,0.347702,0.411003,0.632305


In [42]:
X_train.head()

,ta,tr,vel,rh,pmvg_est,pmvg
13,0.580840,0.456573,0.675392,0.506063,-0.429385,-0.425102
102,0.676664,0.347735,0.148057,0.275927,-0.003960,-0.119385
67,0.624846,0.490115,0.389848,0.507733,0.099354,0.024559
34,0.489663,0.382493,0.628021,0.800314,-0.511791,-0.556970
98,0.848655,0.516771,0.081011,0.708166,0.541918,0.703390


In [43]:
data_teste = X_test
data_teste['pmvg_est'] = final_preds_teste
data_teste['pmvg'] = y_test
data_teste.head()

,ta,tr,vel,rh,pmvg_est,pmvg
33,0.692642,0.333361,0.705758,0.618035,-0.375252,-0.313147
16,0.553586,0.263474,0.591273,0.280329,-0.762058,-0.711406
43,0.446348,0.448418,0.427409,0.350889,-0.411058,-0.292727
129,0.706284,0.235952,0.671317,0.025447,-0.846707,-0.595573
50,0.323094,0.347702,0.411003,0.632305,-0.669160,-0.463924


In [44]:
median_absolute_error(data_teste['pmvg_est'], data_teste['pmvg'])

0.08805795027929432

### Aplicação do modelo em todos os dados e salvando os resultados

In [45]:
data_todos = pd.concat([data_treino, data_teste], sort=False)

In [46]:
data_todos.head()

,ta,tr,vel,rh,pmvg_est,pmvg
13,0.580840,0.456573,0.675392,0.506063,-0.429385,-0.425102
102,0.676664,0.347735,0.148057,0.275927,-0.003960,-0.119385
67,0.624846,0.490115,0.389848,0.507733,0.099354,0.024559
34,0.489663,0.382493,0.628021,0.800314,-0.511791,-0.556970
98,0.848655,0.516771,0.081011,0.708166,0.541918,0.703390


In [47]:
# data_todos.reset_index()
# data_todos.head()

In [48]:
median_absolute_error(data_todos['pmvg_est'], data_todos['pmvg'])

0.080511491044732

In [49]:
# data_todos.index.name = 'id'
# data_todos.reset_index(inplace=True)

In [50]:
res1 = pd.ExcelWriter('res3.xlsx')
data_todos.to_excel(res1,'dados',index=False)
res1.save()

# Desliga cronômetro

In [51]:
fim = perf_counter()
print(fim - inicio)

36.4957784
